This notebook is based on the tensorflow docs: 


https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

adapted for a regression problem.

In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np

In [3]:
file = "/home/isidro/Desktop/chains_server/chains_1000/chains/owaCDM_phy_HD+SN+BBAO+Planck_nested_dynesty_multi_1.txt"

split = 0.8
epochs = 100
nlayers = 2
batch_size = 0

params = np.loadtxt(file, usecols=(2,3,4,5,6))
logL = np.loadtxt(file, usecols=(1))

nparams = len(params)
randomize = np.random.permutation(nparams)
params = params[randomize]
logL = logL[randomize]

maxLogL = np.max(logL)
minLogL = np.min(logL)
ntrain = int(split * nparams)
indx = [ntrain]
# xtrain <- params_training ytrain<-logLtraining
params_training, params_testing = np.split(params, indx)
logL_training, logL_testing = np.split(logL, indx)

In [4]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=1e-4,
                                   patience=5,
                                   restore_best_weights=True)
                                   ]

n_cols = params_training.shape[1]

In [5]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([100, 300, 500]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))


In [6]:
# METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric('loss', display_name="Loss")])  


In [7]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu, input_shape=(n_cols,)),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='mean_squared_error',
        metrics=['mean_squared_error'],
      )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(params_training, logL_training, epochs=epochs, callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)
    
    
    return loss

In [8]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    loss = train_test_model(hparams)
    tf.summary.scalar("loss", loss, step=1)


In [9]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 100, 'dropout': 0.1, 'optimizer': 'Adadelta'}
Epoch 1/100
557/557 [==============================] - 1s 1ms/step - loss: 6784762368.0000 - mean_squared_error: 6784762368.0000
Epoch 2/100
557/557 [==============================] - 1s 1ms/step - loss: 6784765440.0000 - mean_squared_error: 6784765440.0000
Epoch 3/100
557/557 [==============================] - 1s 1ms/step - loss: 6784766464.0000 - mean_squared_error: 6784766464.0000
Epoch 4/100
557/557 [==============================] - 1s 1ms/step - loss: 6784771072.0000 - mean_squared_error: 6784771072.0000
Epoch 5/100
557/557 [==============================] - 1s 1ms/step - loss: 6784765440.0000 - mean_squared_error: 6784765440.0000
Epoch 6/100
140/140 [==============================] - 0s 925us/step - loss: 7101648384.0000 - mean_squared_error: 7101648384.0000
--- Starting trial: run-1
{'num_units': 100, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/100
557/557 [==============================] - 1

557/557 [==============================] - 1s 2ms/step - loss: 6784772608.0000 - mean_squared_error: 6784772608.0000
Epoch 6/100
557/557 [==============================] - 1s 2ms/step - loss: 6784771072.0000 - mean_squared_error: 6784771072.0000
Epoch 7/100
557/557 [==============================] - 1s 1ms/step - loss: 6784766464.0000 - mean_squared_error: 6784766464.0000
Epoch 8/100
557/557 [==============================] - 1s 2ms/step - loss: 6784766976.0000 - mean_squared_error: 6784766976.0000
Epoch 9/100
557/557 [==============================] - 1s 2ms/step - loss: 6784771584.0000 - mean_squared_error: 6784771584.0000
Epoch 10/100
557/557 [==============================] - 1s 1ms/step - loss: 6784770560.0000 - mean_squared_error: 6784770560.0000
Epoch 11/100
557/557 [==============================] - 1s 2ms/step - loss: 6784769536.0000 - mean_squared_error: 6784769536.0000
Epoch 12/100
140/140 [==============================] - 0s 811us/step - loss: 7101652992.0000 - mean_square

557/557 [==============================] - 2s 4ms/step - loss: 6784749568.0000 - mean_squared_error: 6784749568.0000
Epoch 21/100
557/557 [==============================] - 2s 4ms/step - loss: 6784754176.0000 - mean_squared_error: 6784754176.0000
Epoch 22/100
557/557 [==============================] - 3s 6ms/step - loss: 6784749056.0000 - mean_squared_error: 6784749056.0000
Epoch 23/100
557/557 [==============================] - 3s 5ms/step - loss: 6784743424.0000 - mean_squared_error: 6784743424.0000
Epoch 24/100
557/557 [==============================] - 3s 5ms/step - loss: 6784745984.0000 - mean_squared_error: 6784745984.0000
Epoch 25/100
557/557 [==============================] - 3s 5ms/step - loss: 6784748544.0000 - mean_squared_error: 6784748544.0000
Epoch 26/100
557/557 [==============================] - 3s 5ms/step - loss: 6784747520.0000 - mean_squared_error: 6784747520.0000
Epoch 27/100
557/557 [==============================] - 3s 6ms/step - loss: 6784742912.0000 - mean_squa

557/557 [==============================] - 2s 4ms/step - loss: 6784537088.0000 - mean_squared_error: 6784537088.0000
Epoch 84/100
557/557 [==============================] - 2s 4ms/step - loss: 6784532480.0000 - mean_squared_error: 6784532480.0000
Epoch 85/100
557/557 [==============================] - 2s 4ms/step - loss: 6784522752.0000 - mean_squared_error: 6784522752.0000
Epoch 86/100
557/557 [==============================] - 2s 3ms/step - loss: 6784514048.0000 - mean_squared_error: 6784514048.0000
Epoch 87/100
557/557 [==============================] - 2s 3ms/step - loss: 6784509440.0000 - mean_squared_error: 6784509440.0000
Epoch 88/100
557/557 [==============================] - 2s 3ms/step - loss: 6784500736.0000 - mean_squared_error: 6784500736.0000
Epoch 89/100
557/557 [==============================] - 2s 3ms/step - loss: 6784490496.0000 - mean_squared_error: 6784490496.0000
Epoch 90/100
557/557 [==============================] - 2s 3ms/step - loss: 6784484864.0000 - mean_squa

557/557 [==============================] - 2s 4ms/step - loss: 6784765440.0000 - mean_squared_error: 6784765440.0000
Epoch 15/100
557/557 [==============================] - 2s 4ms/step - loss: 6784761856.0000 - mean_squared_error: 6784761856.0000
Epoch 16/100
557/557 [==============================] - 2s 4ms/step - loss: 6784760832.0000 - mean_squared_error: 6784760832.0000
Epoch 17/100
557/557 [==============================] - 2s 3ms/step - loss: 6784758784.0000 - mean_squared_error: 6784758784.0000
Epoch 18/100
557/557 [==============================] - 2s 3ms/step - loss: 6784761856.0000 - mean_squared_error: 6784761856.0000
Epoch 19/100
557/557 [==============================] - 2s 3ms/step - loss: 6784756736.0000 - mean_squared_error: 6784756736.0000
Epoch 20/100
557/557 [==============================] - 2s 3ms/step - loss: 6784759808.0000 - mean_squared_error: 6784759808.0000
Epoch 21/100
557/557 [==============================] - 2s 3ms/step - loss: 6784756736.0000 - mean_squa

557/557 [==============================] - 3s 5ms/step - loss: 6784763904.0000 - mean_squared_error: 6784763904.0000
Epoch 8/100
557/557 [==============================] - 3s 5ms/step - loss: 6784765440.0000 - mean_squared_error: 6784765440.0000
Epoch 9/100
557/557 [==============================] - 3s 5ms/step - loss: 6784757760.0000 - mean_squared_error: 6784757760.0000
Epoch 10/100
557/557 [==============================] - 3s 5ms/step - loss: 6784760832.0000 - mean_squared_error: 6784760832.0000
Epoch 11/100
557/557 [==============================] - 3s 5ms/step - loss: 6784755200.0000 - mean_squared_error: 6784755200.0000
Epoch 12/100
557/557 [==============================] - 3s 5ms/step - loss: 6784757760.0000 - mean_squared_error: 6784757760.0000
Epoch 13/100
557/557 [==============================] - 3s 5ms/step - loss: 6784752128.0000 - mean_squared_error: 6784752128.0000
Epoch 14/100
557/557 [==============================] - 2s 4ms/step - loss: 6784751104.0000 - mean_square

557/557 [==============================] - 2s 4ms/step - loss: 6784249856.0000 - mean_squared_error: 6784249856.0000
Epoch 71/100
557/557 [==============================] - 2s 4ms/step - loss: 6784224256.0000 - mean_squared_error: 6784224256.0000
Epoch 72/100
557/557 [==============================] - 2s 4ms/step - loss: 6784206336.0000 - mean_squared_error: 6784206336.0000
Epoch 73/100
557/557 [==============================] - 2s 4ms/step - loss: 6784183296.0000 - mean_squared_error: 6784183296.0000
Epoch 74/100
557/557 [==============================] - 2s 4ms/step - loss: 6784157184.0000 - mean_squared_error: 6784157184.0000
Epoch 75/100
557/557 [==============================] - 2s 4ms/step - loss: 6784130048.0000 - mean_squared_error: 6784130048.0000
Epoch 76/100
557/557 [==============================] - 2s 4ms/step - loss: 6784106496.0000 - mean_squared_error: 6784106496.0000
Epoch 77/100
557/557 [==============================] - 2s 4ms/step - loss: 6784079872.0000 - mean_squa

140/140 [==============================] - 0s 1ms/step - loss: 1862344960.0000 - mean_squared_error: 1862344960.0000
--- Starting trial: run-10
{'num_units': 500, 'dropout': 0.2, 'optimizer': 'Adadelta'}
Epoch 1/100
557/557 [==============================] - 2s 4ms/step - loss: 6784769024.0000 - mean_squared_error: 6784769024.0000
Epoch 2/100
557/557 [==============================] - 2s 4ms/step - loss: 6784775168.0000 - mean_squared_error: 6784775168.0000
Epoch 3/100
557/557 [==============================] - 2s 4ms/step - loss: 6784771072.0000 - mean_squared_error: 6784771072.0000
Epoch 4/100
557/557 [==============================] - 2s 4ms/step - loss: 6784765440.0000 - mean_squared_error: 6784765440.0000
Epoch 5/100
557/557 [==============================] - 2s 4ms/step - loss: 6784766464.0000 - mean_squared_error: 6784766464.0000
Epoch 6/100
557/557 [==============================] - 2s 4ms/step - loss: 6784762880.0000 - mean_squared_error: 6784762880.0000
Epoch 7/100
557/557 [=

557/557 [==============================] - 2s 4ms/step - loss: 6784570368.0000 - mean_squared_error: 6784570368.0000
Epoch 63/100
557/557 [==============================] - 2s 4ms/step - loss: 6784571904.0000 - mean_squared_error: 6784571904.0000
Epoch 64/100
557/557 [==============================] - 2s 4ms/step - loss: 6784559104.0000 - mean_squared_error: 6784559104.0000
Epoch 65/100
557/557 [==============================] - 2s 4ms/step - loss: 6784552960.0000 - mean_squared_error: 6784552960.0000
Epoch 66/100
557/557 [==============================] - 2s 4ms/step - loss: 6784539648.0000 - mean_squared_error: 6784539648.0000
Epoch 67/100
557/557 [==============================] - 2s 4ms/step - loss: 6784529408.0000 - mean_squared_error: 6784529408.0000
Epoch 68/100
557/557 [==============================] - 2s 4ms/step - loss: 6784522752.0000 - mean_squared_error: 6784522752.0000
Epoch 69/100
557/557 [==============================] - 2s 4ms/step - loss: 6784515072.0000 - mean_squa

557/557 [==============================] - 2s 4ms/step - loss: 2138778624.0000 - mean_squared_error: 2138778624.0000
Epoch 24/100
557/557 [==============================] - 3s 5ms/step - loss: 2126934016.0000 - mean_squared_error: 2126934016.0000
Epoch 25/100
557/557 [==============================] - 3s 5ms/step - loss: 2099508096.0000 - mean_squared_error: 2099508096.0000
Epoch 26/100
557/557 [==============================] - 2s 4ms/step - loss: 2106320512.0000 - mean_squared_error: 2106320512.0000
Epoch 27/100
557/557 [==============================] - 2s 4ms/step - loss: 2137737088.0000 - mean_squared_error: 2137737088.0000
Epoch 28/100
557/557 [==============================] - 3s 5ms/step - loss: 2160373248.0000 - mean_squared_error: 2160373248.0000
Epoch 29/100
557/557 [==============================] - 3s 5ms/step - loss: 2137783296.0000 - mean_squared_error: 2137783296.0000
Epoch 30/100
140/140 [==============================] - 0s 1ms/step - loss: 1970986752.0000 - mean_squa

In [10]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 871), started 1:36:00 ago. (Use '!kill 871' to kill it.)